In [2]:
import numpy as np
import pandas as pd

In [2]:
def check_row(n,i,X):
    if n in X[i]:
        #print ('{} is not a valid entry'.format(n))
        return False
    else:
        return True
    
def check_column(n,j,X):
    if n in X[:,j]:
        #print ('{} is not a valid entry'.format(n))
        return False
    else:
        return True
    
def BOX(X):
    return np.array([[X[3*i:3*(i+1), 3*j:3*(j+1)] for j in range(3)] for i in range(3)])
    
def check_box(n,i,j,X):
    if n in BOX(X)[int(i/3)][int(j/3)]:
        #print ('{} is not a valid entry'.format(n))
        return False
    else:
        return True
    
def check_position(n,i,j,X):
    if n in range(1,10):
        return check_row(n,i,X) and check_column(n,j,X) and check_box(n,i,j,X)
    else:
        return False

def update(n,i,j,Y):
    X=Y.copy()
    if np.isnan(X[i][j])==False:
        print('entry not successful')
        return X
    elif n not in list(range(1,10)):
        #print ('{} is not a valid entry'.format(n))
        return X
    elif check_position(n,i,j,X):
        X[i][j]=n
        return X
    else:
        return X

In [3]:
def possible_entries(i,j,X):
    e=[]
    if np.isnan(X)[i,j]== False:
        return []
    else:
        for n in range(1,10):
            if check_position(n,i,j,X):
                e.append(n)
        return e

In [4]:
def initial_entries(Y):
    X=Y.copy()
    p_e=[[possible_entries(i,j,X) for j in range(9)] for i in range(9)]
    a=81
    m=0
    while np.isnan(X).sum() != a:
        a=np.isnan(X).sum()
        for i in range(9):
            for j in range(9):
                if len(p_e[i][j])==1:
                    X=update(p_e[i][j][0],i,j,X)
                    print('{} put in row: {}, column: {}'.format(p_e[i][j][0],i,j))
        m+=1
        print('{} iteration done'.format(m))
        p_e= [[possible_entries(i,j,X) for j in range(9)] for i in range(9)]
    return X

In [5]:
def index_of_column(n,j,X):
    """
    gives a list of indexes on column j where number n can sit
    """
    a=[]
    for i in range(9):
        if np.isnan(X)[i,j]:
            if n not in X[i]:
                if n not in BOX(X)[int(i/3)][int(j/3)].flatten():
                    a.append(i)
    return a

def index_of_row(n,i,X):
    """
    gives list of indexes on row i where number n can sit
    """
    a=[]
    for j in range(9):
        if np.isnan(X)[i,j]:
            if n not in X[:,j]:
                if n not in BOX(X)[int(i/3)][int(j/3)]:
                    a.append(j)
    return a        

def index_of_box(n,i,j,X):
    """
    gives list of indexes on ith row, jth column box where number n can sit
    """
    a=[]
    box_row= int(i/3)
    box_column= int(j/3)
    box=BOX(X)[box_row][box_column].flatten()
    for k in range(len(box)):
        if np.isnan(X)[i,j]:
            if n not in X[i]:
                if n not in X[:,j]:
                    a.append([3*box_row+int(k/3),3*box_column+k%3])
    return a 

In [6]:
def half_solution(Y):
    X=Y.copy()
    a=81
    m=0
    while a != np.isnan(X).sum():
        a=np.isnan(X).sum()
        for n in range(1,10):
            for j in range(9):
                if len(index_of_column(n,j,X))==1:
                    X=update(n, index_of_column(n,j,X)[0], j, X)
        for n in range(1,10):
            for i in range(9):
                if len(index_of_row(n,i,X))==1:
                    X=update(n, i, index_of_row(n,i,X)[0], X)
                    
        for n in range(1,10):
            for i in range(9):
                for j in range(9):
                    if len(index_of_box(n,i,j,X))==1:
                        l=index_of_box(n,i,j,X)[0]
                        X= update(n,l[0],l[1] ,X)
        print('iteration {} done. Remaining missing values: {} '.format(m+1, np.isnan(X).sum()))
        m+=1
    return X
        

In [7]:
def rec_sol(Y):
    a=81
    n1=0
    X=Y.copy()
    while a != np.isnan(X).sum():
        a=np.isnan(X).sum()
        X=initial_entries(X)
        X=half_solution(X)
        n1 +=1
        print('after {} sets of recurrsion missing value left: {}'.format(n1,np.isnan(X).sum()))
        
    return X

In [43]:
X=rec_sol(hard)
X

1 iteration done
iteration 1 done. Remaining missing values: 54 
iteration 2 done. Remaining missing values: 54 
after 1 sets of recurrsion missing value left: 54
1 put in row: 4, column: 8
1 iteration done
2 iteration done
iteration 1 done. Remaining missing values: 53 
after 2 sets of recurrsion missing value left: 53
1 iteration done
iteration 1 done. Remaining missing values: 53 
after 3 sets of recurrsion missing value left: 53


array([[nan, nan, nan,  7., nan, nan,  5.,  6.,  4.],
       [ 4., nan, nan, nan, nan, nan, nan, nan, nan],
       [nan,  6., nan, nan,  4.,  1., nan,  8., nan],
       [nan,  2.,  4., nan, nan, nan, nan, nan,  6.],
       [ 3., nan,  6., nan, nan, nan, nan,  7.,  1.],
       [ 1., nan, nan, nan,  3., nan,  2., nan,  5.],
       [nan, nan,  3., nan, nan, nan, nan, nan,  9.],
       [nan, nan, nan,  9., nan,  5., nan, nan,  8.],
       [nan,  7., nan, nan,  6., nan, nan,  5., nan]])

In [36]:
l=np.array([[len(possible_entries(i,j,X)) for j in range(9)] for i in range(9)])
l[l==0]=11
l

array([[ 3,  4,  4, 11,  3,  4, 11, 11, 11],
       [11,  5,  6,  5,  4,  5,  4,  4,  3],
       [ 4, 11,  4,  3, 11, 11,  3, 11,  3],
       [ 4, 11, 11,  3,  5,  3,  3,  2, 11],
       [11,  3, 11,  4,  4,  4,  3, 11, 11],
       [11,  2,  3,  3, 11,  5, 11,  2, 11],
       [ 4,  4, 11,  4,  4,  4,  4,  3, 11],
       [ 2,  2,  2, 11,  3, 11,  5,  4, 11],
       [ 3, 11,  4,  5, 11,  4,  3, 11,  2]])

In [37]:
k=l.flatten().argmin()

In [38]:
possible_entries(int(k/9),k%9,rec_sol(X))

1 iteration done
iteration 1 done. Remaining missing values: 53 
after 1 sets of recurrsion missing value left: 53


[3, 9]

In [39]:
possible_entries(0,4,X)

[2, 8, 9]

In [46]:
def trial_or_error(Y, record=[]):
    X= Y.copy()
    #print('copy done')
    a= (np.array([[len(possible_entries(i,j,X)) for j in range(9)] for i in range(9)])==0).sum()
    b= np.isnan(X).sum()
    X=rec_sol(X)
    while (a != 81) :
        #print('enter while loop')
        lentry=np.array([[len(possible_entries(i,j,X)) for j in range(9)] for i in range(9)])
        lentry[lentry==0]=11
        #print(lentry)
        k=lentry.flatten().argmin()
        #print('k={}'.format(k))
        p_e=possible_entries(int(k/9),k%9,X)
        Z=list(X.copy())
        X=update(p_e[0],int(k/9),k%9,X)
        #print('update{},{},{}'.format(p_e[0],int(k/9),k%9))
        X=rec_sol(X)
        record.append([p_e[0],int(k/9),k%9,p_e,Z])
        a= (np.array([[len(possible_entries(i,j,X)) for j in range(9)] for i in range(9)])==0).sum()
        b= np.isnan(X).sum()
        
    if (b !=0):
        return record
    
    elif b==0:
        print('Final Solution is: ')
        return X
        

In [77]:
c=trial_or_error(X)
print(c)

1 iteration done
iteration 1 done. Remaining missing values: 71 
after 1 sets of recurrsion missing value left: 71
1 iteration done
iteration 1 done. Remaining missing values: 70 
after 1 sets of recurrsion missing value left: 70
1 iteration done
iteration 1 done. Remaining missing values: 69 
after 1 sets of recurrsion missing value left: 69
1 iteration done
iteration 1 done. Remaining missing values: 68 
after 1 sets of recurrsion missing value left: 68
1 iteration done
iteration 1 done. Remaining missing values: 67 
after 1 sets of recurrsion missing value left: 67
8 put in row: 5, column: 5
1 iteration done
2 iteration done
iteration 1 done. Remaining missing values: 65 
after 1 sets of recurrsion missing value left: 65
1 iteration done
iteration 1 done. Remaining missing values: 65 
after 2 sets of recurrsion missing value left: 65
1 iteration done
iteration 1 done. Remaining missing values: 64 
after 1 sets of recurrsion missing value left: 64
1 iteration done
iteration 1 done. R

In [64]:
def reconsile (m):
    r=m.copy()
    k=r[-1]
    r=r[:-1]
    puzzle_before_last_error=np.array(k[-1])
    wrong_entry= k[0]
    entry_row=k[1]
    entry_column= k[2]
    possible_entries_on_last_error= k[3]
    p_e=[i for i in possible_entries_on_last_error if i != wrong_entry]
    print(p_e,k,r)
    record=r.copy()
    i=1
    print(i)
    
    if len(p_e)==1:
        print('entered if')
        Z=list(puzzle_before_last_error.copy())
        fam=update(p_e[0], entry_row, entry_column,puzzle_before_last_error)
        print(puzzle_before_last_error)
        fill_and_more=rec_sol(fam)
        record.append([p_e[0], entry_row, entry_column,p_e,Z])
        b=np.isnan(fill_and_more).sum()
        a= (np.array([[len(possible_entries(i,j,fill_and_more)) for j in range(9)] for i in range(9)])==0).sum()
        if b==0:
            print('solution found')
            return fill_and_more
        elif a==81:
            i+=1
            print ('going back one step')
            return reconsile(r)
        else:
            print ('Another branch')
            return trial_or_error(fill_and_more, record)
    else:
        print('entered else')
        Z=list(puzzle_before_last_error.copy())
        fam=update(p_e[0], entry_row, entry_column,puzzle_before_last_error)
        fill_and_more=rec_sol(fam)
        b=np.isnan(fill_and_more).sum()
        a= (np.array([[len(possible_entries(i,j,fill_and_more)) for j in range(9)] for i in range(9)])==0).sum()
        record.append([p_e[0], entry_row, entry_column,p_e,Z])
        if b==0:
            print('solution found')
            return fill_and_more
        elif a==81:
            i+=1
            print('going back a step')
            return reconsile(r)
        else:
            print('Another branch')
            return trial_or_error(fill_and_more,record)

        
    

In [65]:
reconsile(c)

[9] [8, 0, 4, [8, 9], [array([ 2., nan, nan,  7., nan, nan,  5.,  6.,  4.]), array([ 4., nan, nan, nan, nan, nan,  7.,  2., nan]), array([7., 6., 5., 2., 4., 1., 9., 8., 3.]), array([nan,  2.,  4., nan, nan,  7.,  8.,  3.,  6.]), array([ 3., nan,  6., nan, nan, nan,  4.,  7.,  1.]), array([ 1.,  8.,  7., nan,  3., nan,  2.,  9.,  5.]), array([nan, nan,  3., nan, nan, nan,  6.,  4.,  9.]), array([ 6.,  4.,  2.,  9.,  7.,  5.,  3., nan,  8.]), array([nan,  7., nan, nan,  6., nan,  1.,  5.,  2.])]] [[3, 3, 7, [3, 9], [array([nan, nan, nan,  7., nan, nan,  5.,  6.,  4.]), array([ 4., nan, nan, nan, nan, nan, nan, nan, nan]), array([nan,  6., nan, nan,  4.,  1., nan,  8., nan]), array([nan,  2.,  4., nan, nan, nan, nan, nan,  6.]), array([ 3., nan,  6., nan, nan, nan, nan,  7.,  1.]), array([ 1., nan, nan, nan,  3., nan,  2., nan,  5.]), array([nan, nan,  3., nan, nan, nan, nan, nan,  9.]), array([nan, nan, nan,  9., nan,  5., nan, nan,  8.]), array([nan,  7., nan, nan,  6., nan, nan,  5., 

array([[2., 1., 8., 7., 9., 3., 5., 6., 4.],
       [4., 3., 9., 8., 5., 6., 1., 2., 7.],
       [7., 6., 5., 2., 4., 1., 9., 8., 3.],
       [5., 2., 4., 1., 8., 7., 3., 9., 6.],
       [3., 9., 6., 5., 2., 4., 8., 7., 1.],
       [1., 8., 7., 6., 3., 9., 2., 4., 5.],
       [8., 5., 3., 4., 7., 2., 6., 1., 9.],
       [6., 4., 2., 9., 1., 5., 7., 3., 8.],
       [9., 7., 1., 3., 6., 8., 4., 5., 2.]])

In [76]:
import random
X=np.array([[np.nan]*9]*9)
for k in range(10):
    X[random.randint(0,8)][random.randint(0,8)]=random.randint(1,9)
X

array([[nan, nan, nan, nan, nan, nan,  2., nan, nan],
       [nan, nan, nan,  2., nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan,  7.],
       [ 6., nan,  9., nan, nan, nan, nan, nan,  3.],
       [nan,  7., nan,  1., nan, nan, nan, nan, nan],
       [nan,  4., nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan,  1.]])

In [70]:
extreme=np.array([[np.nan,7,np.nan,np.nan,np.nan,4,np.nan,np.nan,np.nan],[np.nan,np.nan,8,np.nan,np.nan,3,2,np.nan,5],[5,np.nan,np.nan,6,np.nan,np.nan,np.nan,np.nan,np.nan],
                  [np.nan,2,np.nan,np.nan,np.nan,1,np.nan,np.nan,np.nan],[np.nan,8,3,np.nan,np.nan,2,1,9,np.nan],[6,np.nan,np.nan,np.nan,np.nan,np.nan,5,2,np.nan],
                  [np.nan,np.nan,np.nan,1,4,np.nan,np.nan,np.nan,8],[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,2],[9,np.nan,np.nan,np.nan,np.nan,np.nan,6,3,1]
                 ])

In [71]:
X=rec_sol(extreme)

1 iteration done
iteration 1 done. Remaining missing values: 54 
iteration 2 done. Remaining missing values: 54 
after 1 sets of recurrsion missing value left: 54
1 iteration done
iteration 1 done. Remaining missing values: 54 
after 2 sets of recurrsion missing value left: 54


In [72]:
X

array([[nan,  7., nan, nan, nan,  4., nan, nan, nan],
       [nan, nan,  8., nan, nan,  3.,  2., nan,  5.],
       [ 5., nan, nan,  6., nan, nan, nan, nan, nan],
       [nan,  2., nan, nan, nan,  1., nan, nan, nan],
       [nan,  8.,  3., nan, nan,  2.,  1.,  9., nan],
       [ 6., nan, nan, nan, nan, nan,  5.,  2., nan],
       [nan, nan, nan,  1.,  4., nan, nan, nan,  8.],
       [ 8., nan, nan, nan, nan, nan, nan, nan,  2.],
       [ 9., nan, nan, nan, nan, nan,  6.,  3.,  1.]])

In [5]:
X.shape==(9,9)

True

In [3]:
X=np.array([[9,2,np.nan,6,1,np.nan,np.nan,np.nan,np.nan],[4,6,np.nan,np.nan,np.nan,3,np.nan,np.nan,np.nan],[np.nan,1,np.nan,7,np.nan,np.nan,9,np.nan,6],
  [np.nan,5,np.nan,np.nan,7,8,2,np.nan,np.nan],[6,7,2,np.nan,np.nan,np.nan,5,8,np.nan],[np.nan,np.nan,np.nan,2,np.nan,6,np.nan,np.nan,3],
   [np.nan,9,6,3,2,np.nan,np.nan,1,4],[7,np.nan,np.nan,np.nan,np.nan,np.nan,6,np.nan,np.nan],[np.nan,np.nan,1,np.nan,6,4,np.nan,9,np.nan]
  ])

In [ ]:
solution(X)

In [ ]:
X1=np.array([[np.nan,np.nan,np.nan,9,np.nan,7,np.nan,3,np.nan],[np.nan,np.nan,8,4,np.nan,3,np.nan,np.nan,np.nan],[np.nan,np.nan,7,np.nan,2,np.nan,np.nan,5,4],
  [9,7,np.nan,3,4,1,5,8,np.nan],[5,8,np.nan,7,np.nan,6,np.nan,np.nan,2],[6,3,np.nan,2,8,5,np.nan,9,1],
   [np.nan,1,5,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],[np.nan,np.nan,np.nan,6,np.nan,np.nan,np.nan,2,np.nan],[np.nan,np.nan,np.nan,5,7,np.nan,np.nan,np.nan,np.nan]
  ])

In [ ]:
solution(X1)

In [66]:
extreme=np.array([[4,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,1,np.nan],[np.nan,np.nan,np.nan,4,np.nan,2,3,np.nan,np.nan],[8,3,6,np.nan,1,np.nan,np.nan,np.nan,np.nan],
                  [2,np.nan,np.nan,np.nan,6,np.nan,np.nan,5,7],[np.nan,9,np.nan,5,np.nan,np.nan,6,np.nan,1],[np.nan,np.nan,7,1,np.nan,np.nan,np.nan,np.nan,np.nan],
                  [np.nan,np.nan,np.nan,np.nan,8,6,np.nan,np.nan,3],[7,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],[6,4,np.nan,np.nan,7,np.nan,np.nan,np.nan,2]
                 ])

In [ ]:
solution(extreme)

In [9]:
hard=np.array([
    [np.nan,np.nan,np.nan,7,np.nan,np.nan,5,6,np.nan],[4,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],[np.nan,6,np.nan,np.nan,4,1,np.nan,8,np.nan],
    [np.nan,2,4,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],[3,np.nan,6,np.nan,np.nan,np.nan,np.nan,7,np.nan],[1,np.nan,np.nan,np.nan,3,np.nan,2,np.nan,5],
    [np.nan,np.nan,3,np.nan,np.nan,np.nan,np.nan,np.nan,9],[np.nan,np.nan,np.nan,9,np.nan,5,np.nan,np.nan,8],[np.nan,7,np.nan,np.nan,6,np.nan,np.nan,5,np.nan]
    
])

In [ ]:
solution(hard)

In [ ]:
medium=np.array([
    [np.nan,6,np.nan,4,np.nan,np.nan,np.nan,9,2],[np.nan,2,5,np.nan,np.nan,np.nan,4,np.nan,np.nan],[7,4,np.nan,2,3,9,np.nan,np.nan,np.nan],
    [5,9,6,np.nan,2,np.nan,np.nan,np.nan,1],[4,np.nan,np.nan,np.nan,np.nan,5,np.nan,7,np.nan],[2,8,7,np.nan,4,np.nan,np.nan,np.nan,np.nan],
    [np.nan,7,np.nan,8,np.nan,2,np.nan,1,5],[8,np.nan,np.nan,5,np.nan,7,3,np.nan,np.nan],[np.nan,5,np.nan,np.nan,np.nan,4,np.nan,np.nan,np.nan]
])

In [ ]:
solution(medium)

In [ ]:
X=np.array([[9,2,np.nan,6,1,np.nan,np.nan,np.nan,np.nan],[4,6,np.nan,np.nan,np.nan,3,np.nan,np.nan,np.nan],[np.nan,1,np.nan,7,np.nan,np.nan,9,np.nan,6],
  [np.nan,5,np.nan,np.nan,7,8,2,np.nan,np.nan],[6,7,2,np.nan,np.nan,np.nan,5,8,np.nan],[np.nan,np.nan,np.nan,2,np.nan,6,np.nan,np.nan,3],
   [np.nan,9,6,3,2,np.nan,np.nan,1,4],[7,np.nan,np.nan,np.nan,np.nan,np.nan,6,np.nan,np.nan],[np.nan,np.nan,1,np.nan,6,4,np.nan,9,np.nan]
  ])

In [ ]:
solution(X)